# Two-Particle Self-Consistent approach (TPSC) tutorial

This tutorial is done in six steps

1. You will first learn how to manipulate multivariable Green's functions. 
   You will also convince yourself that for nearest-neighbor non-interacting models, the Fermi surface has perfect    nesting(TPSC-1)

2. You will compute the Lindhard function for the non-interacting susceptibility (TPSC-2)

3. You will then compute the RPA approximation to check the divergence at ($\pi,\pi)$ (TPSC-3)

4. Renormalized spin and charge vertices in TPSC are computed. RPA does not satisfy the Pauli principle (TPSC-4)
   
5. The spin susceptibility is computed to show that it does not diverge at finite temperature. (TPSC-5)

6. A challenging exercice with the self-energy (hot spots) is left at the end if you have time. (TPSC-6)

## Lattice Green function

In this notebook, we will first manipulate the Green's function on a square lattice with nearest neighbour hopping $t$, 

\begin{equation}
G_0(\mathbf{k},i\omega_n)=\frac{1}{i\omega_n  + \mu - \epsilon(\mathbf{k})}
\end{equation}

whose dispersion is $\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y})$, where $\mathbf{k}$ is a vector in the Brillouin zone (in units where the lattice spacing is unity $a=1$) and $i\omega_n$ is a Matsubara frequency.

In [ ]:
# Imports 
%matplotlib inline
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import Gf, MeshProduct, MeshBrillouinZone, MeshImFreq, Idx
from pytriqs.plot.mpl_interface import plt
import numpy as np
import warnings 
warnings.filterwarnings("ignore") #ignore some matplotlib warnings
from math import cos, pi
plt.rcParams["figure.figsize"] = (10,9) # set default size for all figures

In [ ]:
# Regroup some parameters of the computation used later
beta = 1/0.4 # Inverse temperature
t = 1.0      # Hopping   
n_k = 128    # Number of points in the Brillouin Zone mesh (for each dimension)
n_w = 128    # Number of Matsubara frequencies
mu = 0       # Chemical potential

## A mesh on a Brillouin Zone

We first define a simple Bravais lattice (`BravaisLattice`) in 2 dimensions with basis vectors $\hat{e}_x = (1, 0, 0)$ and $\hat{e}_y=(0, 1, 0)$, and given the bravais lattice we construct the reciprocal (momentum) space Brillouin zone (`BrillouinZone`).

In [ ]:
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)]) # Two unit vectors in R3
BZ = BrillouinZone(BL) 

The complex valued Green's function $G(\mathbf{k}, i\omega_n) $ is represented on the cartesian product mesh : $ (\mathbf{k} \times i\omega_n) \rightarrow {\mathcal{C}}$. 

We construct $G$ by first defining the two separate meshes in momentum space $\mathbf{k}$ (`MeshBrillouinZone`) and frequency space $i\omega_n$ (`MeshImFreq`) and then use the `MeshProduct` of these meshes as the mesh for $G(\mathbf{k}, i\omega_n)$.

In [ ]:
kmesh = MeshBrillouinZone(BZ, n_k=n_k)
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

g0 = Gf(mesh=MeshProduct(kmesh, wmesh), target_shape=[])  # g0(k,omega), scalar valued

To fill the Green's function we construct a function for the dispersion $\epsilon(\mathbf{k})$ and set each element of $G$ by looping over the momentum and frequency meshes.

In [ ]:
#%%timeit
def eps(k):
    return -2 * t* (cos(k[0]) + cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g0.mesh[0]:
    for w in g0.mesh[1]:
        g0[k, w] = 1/(w - eps(k))

## (Optional) More advanced

The python is quite slow, as any serious (double !) loop in Python.

It is a good opportunity to illustrate a bit the C++ layer of the TRIQS library, 
using the TRIQS/cpp2py tool to wrap Python and C++ in a simple case.

We first import an ipython "magic" command `%%cpp2py` :

In [ ]:
%reload_ext cpp2py.magic

The function `compute_g0` below is written in C++. 
It takes a Green's function, and the hopping $t$
and does the same computation as above.

When executing the "magic" cell, the C++ is analysed by the Clang compiler, 
a wrapping code (to expose it to Python and convert its argument) is generated,
compiled. The whole is linked into a module, which is loaded automatically,
so the `compute_g0` function is available in Python.

`NB : We could show how to acces g0.data to vectorize its initialization`

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

void compute_g0(gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued> g, double t) {
      for (auto [k,w] : g.mesh()) // loop on points
       g[k,w] = 1/(w - (-2)*t *(cos(k[0]) + cos(k[1]))); 
       // k,w are points on the grid and cast in points on the k mesh, and a Matsubara frequency resp.
}

In [ ]:
#%%timeit
# we call the function and check that we get the same answer...
g0b = Gf(mesh=MeshProduct(kmesh, wmesh), target_shape=[])  # g0(k,omega), scalar valued
compute_g0(g0b,t)

assert np.amax(np.abs(g0b.data - g0.data)) < 1.e-14, "It should be the same ..."

## Save Green function for later use

In [ ]:
from pytriqs.archive import HDFArchive
with HDFArchive("tpsc.h5") as R:
    R['g0_kw'] = g0

## Evaluate the Green function

The Green function $g_0(k,i\omega_n)$ can be evaluated for :

- $k$ is a vector of double 
- $n$ is an integer, the $n$ in $i\omega_n$

The result will be a linear interpolation on the Brillouin zone 
  with the closest points of $k$ on the grid of $g_0$.

One can use $g_0$ as any python function of $k$ and $i\omega_n$, 
and forget its precise representation in memory (what is the grid, etc...).
This will allows to simply write the plot. 


Example:

In [ ]:
print g0((pi,pi,0), 2)

## For nearest-neighbor model, the Fermi surface is nested

### Plot of the momentum distribution curve at the Fermi level

The Fermi surface is nested. 

    What do we mean by that?
    What is the nesting vector?

In [ ]:
# take a simple numpy grid (independant of the actual grid of g0)
kgrid1d = np.linspace(-np.pi, np.pi, n_k + 1, endpoint=True)  # a linear grid
kx, ky = np.meshgrid(kgrid1d, kgrid1d)                      # a 2d grid of points from numpy 

# To make the matplotlib plot, we need a function kx, ky -> real
# so we quickly make two simple ones...

# The spectral function vs k at \omega_0
spectral = lambda kx, ky: -g0( (kx,ky,0), 0).imag / pi
# The denominator that should vanish at the location of the Fermi surface.
fs = lambda kx, ky: (1/g0( (kx,ky,0) , 0)).real

# make the color plot
plt.figure(figsize=(7,7))
plt.pcolor(kx, ky, np.vectorize(spectral)(kx,ky))
plt.colorbar()
plt.contour(kx, ky, np.vectorize(fs)(kx,ky), levels=[0], colors='white')
plt.axes().set_aspect('equal')

# Cosmetics
plt.xticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])    
plt.yticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])
plt.xlabel(r"$k_x$"); plt.ylabel(r"$k_y$")
plt.title("Momentum distribution curve (MDC) at the Fermi level");

## Partial evaluation

Given a function $g(k,\omega)$, for a given $k_0$, it is possible to obtain
the function $\omega \rightarrow g(k_0, \omega)$ :


In [ ]:
k0 = (0.02,0.01,0)      # a k point as a tuple of 3 floats
gw = g0(k0, all)        # We use the "built-in" function all here as equivalent of :, 
                        # which Python does not permit in ()
                        # gw is now a function of $\omega$ only
                        # The result is a linear interpolation in k0
            
assert gw.mesh == g0.mesh[1]            # The meshes in indeed a simple ImFreq mesh now
assert abs(gw(0) - g0(k0, 0)) < 1.e-14  # Partially evaluate the function and use it, or directly evaluate is the same

The new function is interpolated linearly for the point $k_0$ on the original Brillouin zone grid.

## Momentum distribution, $n_k$
Using the previous method, plot $n(k)$ along the diagonal of the Brillouin Zone to
see the Fermi surface


In [ ]:
plt.figure(figsize=(15,7))

# function k-> g0(k, :) and take the density, a function seen in a previous tutorial (GfImFreq). 
# You are plotting the occupation probability in momentumm state k
n_of_k = lambda k: g0 ( (k, k,0),all).density().real
plt.plot(kgrid1d/pi, np.vectorize(n_of_k)(kgrid1d), '-o')

# Internal check, not part of the question
# We replot using the point *on the grid* directly.  Idx(k_idx,k_idx,0) is the index of points on the grid 
if 1: 
    d = lambda k_idx: g0[Idx(k_idx,k_idx,0),:].density().real 
    kr = range(n_k/2)
    plt.plot([x /(n_k/2.0) for x in kr], np.vectorize(d)(kr), '-x')
